# This Looks Like That There

Main training notebook.

In [1]:
import os
import sys
import time
import imp #imp.reload(module)

import numpy as np
from tqdm import trange
from icecream import ic

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import tensorflow as tf

import network
import experiment_settings 
import data_functions
import push_prototypes
import plots
import common_functions

In [2]:
__author__ = "Elizabeth A. Barnes and Randal J Barnes"
__version__ = "24 November 2021"

In [3]:
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi']= 150
dpiFig = 300.

## Print the detailed system info

In [4]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"tensorflow version = {tf.__version__}")

python version = 3.9.4 (default, Apr  9 2021, 09:32:38) 
[Clang 10.0.0 ]
numpy version = 1.20.1
tensorflow version = 2.5.0


## Define experiment settings and directories

In [5]:
EXP_NAME = 'quadrants'#'mjo'

imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

imp.reload(common_functions)
model_dir, model_diagnostics_dir, vizualization_dir = common_functions.get_exp_directories(EXP_NAME)

## Define the network parameters

In [6]:
RANDOM_SEED          = settings['random_seed']
BATCH_SIZE_PREDICT   = settings['batch_size_predict']
BATCH_SIZE           = settings['batch_size']
NLAYERS              = settings['nlayers']
NFILTERS             = settings['nfilters']   
assert(len(NFILTERS)==NLAYERS)

NCLASSES             = settings['nclasses']
PROTOTYPES_PER_CLASS = settings['prototypes_per_class']
NPROTOTYPES          = np.sum(PROTOTYPES_PER_CLASS)

NEPOCHS              = settings['nepochs']
LR_INIT              = settings['lr']
LR_CALLBACK_EPOCH    = settings['lr_cb_epoch']
PATIENCE             = 100

## Initialize

In [7]:
np.random.seed(RANDOM_SEED)
rng = np.random.default_rng(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## Get and process the data

In [8]:
imp.reload(data_functions)
DATA_NAME = settings['data_name']
DATA_DIR = settings['data_dir']

if(EXP_NAME[:3]=='mjo'):

    labels, data, lat, lon, time = data_functions.load_mjo_data(DATA_DIR)
    X_train, y_train, time_train, X_val, y_val, time_val, X_test, y_test, time_test = data_functions.get_and_process_mjo_data(labels,
                                                                                         data,
                                                                                         time,
                                                                                         rng, 
                                                                                         colored=settings['colored'],
                                                                                         standardize=settings['standardize'],
                                                                                         shuffle=settings['shuffle'],
                                                                                        )        
elif(EXP_NAME[:9]=='quadrants'):
    filename = DATA_DIR + DATA_NAME + '.mat'
    X_train, y_train, X_val, y_val, X_test, y_test, lat, lon = data_functions.get_and_process_data(filename, 
                                                                                        rng, 
                                                                                        colored=settings['colored'],
                                                                                        standardize=settings['standardize'],
                                                                                        shuffle=settings['shuffle'],
                                                                                        )      
    

loading ./data/data_quadrants.mat
shuffling the data before train/validation/test split.
raw_data        = (9000, 100, 100, 1)
training data   = (7200, 100, 100, 1), (7200,)
validation data = (1800, 100, 100, 1), (1800,)
test data       = (0, 100, 100, 1), (0,)
X_mean          = -0.00016575411048148377
X_std           = 0.11273003191226523


In [9]:
proto_class_mask = network.createClassIdentity(PROTOTYPES_PER_CLASS)

prototypes_of_correct_class_train = np.zeros((len(y_train),NPROTOTYPES))
for i in range(0,prototypes_of_correct_class_train.shape[0]):
    prototypes_of_correct_class_train[i,:] = proto_class_mask[:,int(y_train[i])]
    
prototypes_of_correct_class_val   = np.zeros((len(y_val),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_val.shape[0]):
    prototypes_of_correct_class_val[i,:] = proto_class_mask[:,int(y_val[i])]

prototypes_of_correct_class_test   = np.zeros((len(y_test),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_test.shape[0]):
    prototypes_of_correct_class_test[i,:] = proto_class_mask[:,int(y_test[i])]
    

## Define the training callbacks and metrics

In [10]:
# callbacks
def scheduler(epoch, lr):
    if epoch < LR_CALLBACK_EPOCH:
        return np.round(lr,8)
    else:
        if(epoch % 2 == 0):
            return lr/2.
        else:
            return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)    
    
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    mode='max',
    patience=settings['patience'], 
    restore_best_weights=True, 
    verbose=1
)

callbacks_list = [
#     lr_callback,
#     es_callback,
]            

# metrics
metrics_list = [
    tf.keras.metrics.SparseCategoricalAccuracy(),
]

## Instantiate the model

In [11]:
__ = imp.reload(network)
tf.keras.backend.clear_session()

model = network.build_model(
    nlayers              = NLAYERS,
    nfilters             = NFILTERS,
    input_shape          = X_train.shape[1:],
    output_shape         = NCLASSES,
    prototypes_per_class = PROTOTYPES_PER_CLASS,
    network_seed         = RANDOM_SEED,    
    prototype_channels   = settings['prototype_channels'],    
    coeff_cluster        = settings['coeff_cluster'],
    coeff_separation     = settings['coeff_separation'],
    coeff_l1             = settings['coeff_l1'],
    incorrect_strength   = settings['incorrect_strength'],
    double_conv          = settings['double_conv'],
    kernel_l1_coeff      = 0.0,#settings['kernel_l1_coeff'],
    kernel_l2_coeff      = 0.0,#settings['kernel_l2_coeff'],
    drop_rate            = 0.0,
    drop_rate_final      = 0.0,        
    
)
model.summary()

2
[32, 32]
(100, 100, 1)
3
[5, 5, 5]
88
False
0.17197201619672103
-0.017197201619672104
0.5
-0.5
False
8
128
0.0
0.0
0.0
0.0
False
Model: "full_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 320         inputs[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 100, 100, 32) 0           conv_0[0][0]                     
__________________________________________________________________________________________________
maxpooling_0 (AveragePooling2D) (None, 50, 50, 32)   0   

## Load pre-trained weights into convolutional layers

In [12]:
if(settings['pretrain'] == True):

    if(settings['pretrain_exp'] is None):
        PRETRAINED_MODEL = model_dir + 'pretrained_model_' + EXP_NAME 
    else:
        PRETRAINED_MODEL = './saved_models/' + settings['pretrain_exp'] 

    print('loading pretrained convolutional layers from ' + PRETRAINED_MODEL)
    pretrained_model = tf.keras.models.load_model(PRETRAINED_MODEL)

    for layer in range(1,len(model.layers)):
        if(model.layers[layer].name[:4]=='conv'):
            print('   loading pretrained weights for --> ' + model.layers[layer].name)
            model.layers[layer].set_weights(pretrained_model.layers[layer].get_weights())
else:
    print('no pretrained model specified. keeping random initialized weights.')
    

loading pretrained convolutional layers from ./saved_models/quadrants/pretrained_model_quadrants
   loading pretrained weights for --> conv_0
   loading pretrained weights for --> conv_1


In [13]:
# raise ValueError('here')

***

# Run Training Stages

In [14]:
imp.reload(network)
imp.reload(plots)
imp.reload(push_prototypes)
imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

In [15]:
ic(np.shape(X_train))
ic(np.shape(prototypes_of_correct_class_train))
ic(np.shape(prototypes_of_correct_class_train))

ic| np.shape(X_train): (7200, 100, 100, 1)
ic| np.shape(prototypes_of_correct_class_train): (7200, 15)
ic| np.shape(prototypes_of_correct_class_train): (7200, 15)


(7200, 15)

In [16]:
imp.reload(push_prototypes)
NEPOCHS    = settings['nepochs']
STAGE_LIST = (0,1,2,3,4,5,6,7,8,9)#range(len(NEPOCHS))#(1,2,3,4,5)#range(len(NEPOCHS))

for stage in STAGE_LIST:
    
    print('--------------------')
    print('TRAINING STAGE = ' + str(stage))
    print('--------------------')

    # load previously trained stage, unless it is the 0th stage
    if(stage != 0):
        tf.keras.backend.clear_session()
        model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage-1)
#         model = common_functions.load_model(model_filename)
        model.load_weights(model_filename)
        
    # learn layers (during even numbered stages)
    if(stage % 2 == 0):
        # train prototypes layers (and possibly CNN layers)
        if(settings['pretrain']==False and settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(settings['train_cnn_in_stage'] == False or stage==0):
            model = network.set_trainable_layers(model, [False,True,True,False])
        elif(settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(stage >= settings['train_cnn_in_stage']):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        else:
            model = network.set_trainable_layers(model, [False,True,True,False])
    else:
        #.......................................................
        # push the prototypes
        #.......................................................        
        model, push_info = push_prototypes.push(model, 
                                                [X_train,prototypes_of_correct_class_train], 
                                                prototypes_of_correct_class_train, 
                                                perform_push=True,
                                                batch_size=BATCH_SIZE_PREDICT,
                                                verbose=False,
                                               )        
        print('Push complete.\n')            

        # train weights layer only
        model = network.set_trainable_layers(model, [False,False,False,True])        

    #.......................................................
    # compile the model
    #.......................................................
    if(stage>=settings['cut_lr_stage']):
        lr_factor = 10.**(np.floor((stage-settings['cut_lr_stage']+2)/2))
    else:
        lr_factor = 1.
    if(LR_INIT/lr_factor<settings['min_lr']):
        lr_factor = LR_INIT/settings['min_lr']
    print('learning rate = ' + str(np.asarray(LR_INIT/lr_factor,dtype='float32')))

    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=np.asarray(LR_INIT/lr_factor,dtype='float32'), 
        ),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics = metrics_list,
    )
#     model.summary()
    ic(np.min(model.layers[-3].get_weights()[1]),np.max(model.layers[-3].get_weights()[1]))

    #.......................................................
    # train the model
    #.......................................................
    print('Training the model...')    
    
    tf.random.set_seed(RANDOM_SEED)   
    np.random.seed(RANDOM_SEED)    
    history = model.fit(
        [X_train,prototypes_of_correct_class_train],
        y_train,
        validation_data=([[X_val,prototypes_of_correct_class_val]], [y_val]),
        batch_size=BATCH_SIZE,
        epochs=NEPOCHS[stage],
        shuffle=True,
        verbose=1,
        callbacks=callbacks_list
    )
    print('Training complete.\n')            
        

    # save the model at this training stage
    model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage)
    common_functions.save_model(model, model_filename) 
    
    #.......................................................
    # plot results
    #.......................................................  
    try:
        # plot loss history of the model
        plots.plot_loss_history(history)
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_loss_history_stage' + str(stage) + '.png', dpi=dpiFig)    
        plt.close()

        # plot the weights
        plots.plot_weights(model, PROTOTYPES_PER_CLASS)    
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_weights_stage' + str(stage) + '.png', dpi=dpiFig)
        plt.close()
    except:
        print('not making plots...')
        plt.close()

ic| np.min(model.layers[-3].get_weights()[1]): 0.0
    np.max(model.layers[-3].get_weights()[1]): 0.0


--------------------
TRAINING STAGE = 0
--------------------
   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 0.01
Training the model...
Epoch 1/10
225/225 [==============================] - 24s 102ms/step - loss: 8.6444 - sparse_categorical_accuracy: 0.5819 - cluster_cost: 1.8234 - separation_cost: 1.6761 - l1_weights_cost: 7.5000 - val_loss: 8.1004 - val_sparse_categorical_accuracy: 0.7456 - val_cluster_cost: 0.4206 - val_separation_cost: 0.4613 - val_l1_weights_cost: 7.5000
Epoch 2/10
225/225 [==============================] - 22s 97ms/step - loss: 7.8084 - sparse_categorical_accuracy: 0.8993 - cluster_cost: 0.2414 - separation_cost: 0.3008 - l1_weights_cost: 7.5000 - val_loss: 7.7118 - val_sparse_categorical_accuracy: 0.9356 - val_cluster_cost: 0.2010 - val_separation_cost: 0.2278 - val_l1_weights_cost: 7.5000
Epoch 3/10


ic| np.min(model.layers[-3].get_weights()[1]): -0.77609557
    np.max(model.layers[-3].get_weights()[1]): 2.7403998


Training the model...
Epoch 1/10
225/225 [==============================] - 15s 62ms/step - loss: 3.1572 - sparse_categorical_accuracy: 0.8827 - cluster_cost: 0.0026 - separation_cost: 8.1787e-05 - l1_weights_cost: 1.8310 - val_loss: 0.7290 - val_sparse_categorical_accuracy: 0.8861 - val_cluster_cost: 0.0025 - val_separation_cost: 7.0672e-05 - val_l1_weights_cost: 0.0723
Epoch 2/10
225/225 [==============================] - 14s 61ms/step - loss: 0.6271 - sparse_categorical_accuracy: 0.8885 - cluster_cost: 0.0026 - separation_cost: 8.1787e-05 - l1_weights_cost: 0.0531 - val_loss: 0.4811 - val_sparse_categorical_accuracy: 0.9078 - val_cluster_cost: 0.0025 - val_separation_cost: 7.0672e-05 - val_l1_weights_cost: 0.0390
Epoch 3/10
225/225 [==============================] - 15s 65ms/step - loss: 0.4462 - sparse_categorical_accuracy: 0.9046 - cluster_cost: 0.0026 - separation_cost: 8.1787e-05 - l1_weights_cost: 0.0473 - val_loss: 0.4391 - val_sparse_categorical_accuracy: 0.9172 - val_cluster

ic| np.min(model.layers[-3].get_weights()[1]): -0.77609557
    np.max(model.layers[-3].get_weights()[1]): 2.7403998


...

Training the model...
Epoch 1/10
225/225 [==============================] - 14s 59ms/step - loss: 0.5137 - sparse_categorical_accuracy: 0.9548 - cluster_cost: 2.6177e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.1804 - val_loss: 0.5035 - val_sparse_categorical_accuracy: 0.9411 - val_cluster_cost: 2.6234e-05 - val_separation_cost: 0.0000e+00 - val_l1_weights_cost: 0.0948
Epoch 2/10
225/225 [==============================] - 13s 58ms/step - loss: 0.3347 - sparse_categorical_accuracy: 0.9513 - cluster_cost: 2.6177e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0620 - val_loss: 0.3996 - val_sparse_categorical_accuracy: 0.9417 - val_cluster_cost: 2.6234e-05 - val_separation_cost: 0.0000e+00 - val_l1_weights_cost: 0.0527
Epoch 3/10
225/225 [==============================] - 13s 59ms/step - loss: 0.2819 - sparse_categorical_accuracy: 0.9553 - cluster_cost: 2.6177e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0404 - val_loss: 0.3258 - val_sparse_categorical_accuracy: 

ic| np.min(model.layers[-3].get_weights()[1]): -2.1521041
    np.max(model.layers[-3].get_weights()[1]): 5.0507536


--------------------
TRAINING STAGE = 8
--------------------
   conv_0 --> True
   maxpooling_0 --> True
   conv_1 --> True
   maxpooling_1 --> True
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 1e-04
Training the model...
Epoch 1/10
225/225 [==============================] - 39s 171ms/step - loss: 0.1668 - sparse_categorical_accuracy: 0.9608 - cluster_cost: 1.5164e-04 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0352 - val_loss: 0.1953 - val_sparse_categorical_accuracy: 0.9611 - val_cluster_cost: 3.0785e-04 - val_separation_cost: 0.0000e+00 - val_l1_weights_cost: 0.0352
Epoch 2/10
225/225 [==============================] - 40s 178ms/step - loss: 0.1449 - sparse_categorical_accuracy: 0.9669 - cluster_cost: 4.1545e-04 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0352 - val_loss: 0.1740 - val_sparse_categorical_accuracy: 0.9639 - val_cluster_cost: 6.1035e-04 - val_separation_cost: 0.0000e+00 - val_l1_we

ic| np.min(model.layers[-3].get_weights()[1]): -2.1529968
    np.max(model.layers[-3].get_weights()[1]): 5.0716286


Training the model...
Epoch 1/10
225/225 [==============================] - 16s 67ms/step - loss: 0.1859 - sparse_categorical_accuracy: 0.9631 - cluster_cost: 3.6214e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0244 - val_loss: 0.1950 - val_sparse_categorical_accuracy: 0.9594 - val_cluster_cost: 3.4265e-05 - val_separation_cost: 0.0000e+00 - val_l1_weights_cost: 0.0200
Epoch 2/10
225/225 [==============================] - 14s 62ms/step - loss: 0.1814 - sparse_categorical_accuracy: 0.9607 - cluster_cost: 3.6214e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0195 - val_loss: 0.1942 - val_sparse_categorical_accuracy: 0.9567 - val_cluster_cost: 3.4265e-05 - val_separation_cost: 0.0000e+00 - val_l1_weights_cost: 0.0180
Epoch 3/10
225/225 [==============================] - 15s 66ms/step - loss: 0.1801 - sparse_categorical_accuracy: 0.9607 - cluster_cost: 3.6214e-05 - separation_cost: 0.0000e+00 - l1_weights_cost: 0.0180 - val_loss: 0.1882 - val_sparse_categorical_accuracy: 